### Monte Carlo Methods

In this notebook the implementation of Monte Carlo algorithm is provided using the Open AI gym Blackjack environment.

In [1]:
# imports
import sys
from collections import defaultdict
import gym
import numpy as np


In [2]:
# create environment
env = gym.make('Blackjack-v1', new_step_api=True)

Every state of the blackjack env is a tuple of 
* the player's current total points $\in \{0,1,\dots, 31\}$,
* the dealer's face up card $\in \{1, \dots, 10\}$
* Whether the player has a usable ace or not (`no`=0, `yes`=1)

The agent can choose from two actions:
```
    STICK = 0
    HIT = 1
```


### Monte Carlo Methods

Monte Carlo methods are a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. They are often used in physical and mathematical problems and are most useful when it is difficult or impossible to compute an exact result with a deterministic algorithm.

### Mathematical Formulation

The goal of Monte Carlo methods in reinforcement learning is to estimate the value of a policy. The value of a state $ s $ under a policy $ \pi $ is defined as:

$$ V^\pi(s) = \mathbb{E}_\pi \left[ \sum_{t=0}^{\infty} \gamma^t R_{t+1} \mid S_0 = s \right] $$

where:
- $ V^\pi(s) $ is the value of state $ s $ under policy $ \pi $
- $ \mathbb{E}_\pi $ denotes the expected value given that the agent follows policy $ \pi $
- $ \gamma $ is the discount factor
- $ R_{t+1} $ is the reward received at time step $ t+1 $
- $ S_0 $ is the initial state

#### Monte Carlo Prediction

Monte Carlo prediction methods estimate the value of a policy by averaging the returns observed after visits to each state. According to the law of large numbers, the average should eventually converge to the expected return. The return $ G_t $ is the total discounted reward from time step $ t $ onward:

$$ G_t = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1} $$

We can calculate the mean of a sequence $x_1, x_2, \dots$ incrementally:

$$
    \begin{align}
        \mu_k & = \frac{1}{k} \sum_{i=1}^{k} x_i \notag\\
        & = \frac{1}{k} (x_k + \sum_{i=1}^{k-1} x_i) \notag\\
        & = \frac{1}{k} (x_k + (k-1) \mu_{k-1}) \notag\\
        & = \mu_{k-1} + \frac{1}{k}(x_k - \mu_{k-1}) \notag
    \end{align}
$$

Using that we can calculate the value of state $ V(s) $ as the average of the returns observed after visits to $ s $:

$ V(s) \leftarrow V(s) + \frac{1}{N(s)} \left( G_t - V(s) \right) $, where is the counter for state-visitation.

We can use step size $\alpha$ instead to calculate the running mean.

$ V(s) \leftarrow V(s) + \alpha \left( G_t - V(s) \right) $, where $ \alpha $ is the step-size parameter.

<div style="text-align: center;">
    <img src="../resources/first_visit_mc_prediction.png" alt="Alt text" width="500"/>
    <figcaption>Pseudocode for first visit Monte Carlo Prediction</figcaption>
</div>

In [ ]:
def generate_stochastic_episode(env, probs=[0.8, 0.2]):
    """
    Generate a stochastic episode given a probability distribution over actions.
    """
    episode = []
    state = env.reset()
    done = False
    while not done:
        probs = [0.8, 0.2] if state[0] < 18 else [0.2, 0.8]
        action = np.random.choice(np.arange(2), p=probs)
        next_state, reward, done, _, _ = env.step(action)
        episode.append((next_state, action, reward))
    return episode

In [61]:
def monte_carlo_prediction(env, num_episodes, generate_episode, gamma=1.0):
    """
    Monte Carlo prediction algorithm.
    """
    # initialize empty dictionaries of arrays
    returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
    N = defaultdict(lambda: np.zeros(env.action_space.n))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))

    for i_episode in range(num_episodes):
        # check the progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        # generate an episode
        episode = generate_episode(env)
        states, actions, rewards = zip(*episode)
        # prepare for discounting
        discounts = np.array([gamma**i for i in range(len(episode)+1)])
        # update the sum of the returns, number of visits, and action-value 
        # function estimates for each state-action pair in the episode
        for i, state in enumerate(states):
            returns_sum[state][actions[i]] += sum(rewards[i:]*discounts[:-(i+1)])
            N[state][actions[i]] += 1.0
            Q[state][actions[i]] = returns_sum[state][actions[i]] / N[state][actions[i]]
    return Q

In [62]:
Q = monte_carlo_prediction(env, 500000, generate_stochastic_episode)

Episode 499000/500000.

#### Monte Carlo Control

Monte Carlo control methods aim to find the optimal policy by iteratively improving the policy based on the value estimates. The action-value function $ Q(s, a) $ is defined as the expected return starting from state $ s $, taking action $ a $, and thereafter following policy $ \pi $:

$$ Q^\pi(s, a) = \mathbb{E}_\pi \left[ \sum_{t=0}^{\infty} \gamma^t R_{t+1} \mid S_0 = s, A_0 = a \right] $$

The policy is then improved by choosing the action that maximizes the action-value function:

$$ \pi(s) = \arg\max_a Q(s, a) $$

This process is repeated until the policy converges to the optimal policy $ \pi^* $.

#### Conclusion

Monte Carlo methods are powerful tools for solving reinforcement learning problems. They are particularly useful when the model of the environment is unknown or difficult to compute. By leveraging random sampling and averaging, Monte Carlo methods provide a way to estimate the value of policies and find the optimal policy through iterative improvement.